In [1]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
import nltk
import pandas as pd
import re
import time
import torch
import torch.nn as nn

nltk.download('stopwords')
encoder = LabelEncoder()

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /common/home/ac1771/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Good to go!


This code uses the Huffington Post News Category dataset from kaggle which can be found here: [Source](https://www.kaggle.com/datasets/rmisra/news-category-dataset)

In [3]:
class MyDataset(Dataset):
    def __init__(self,file_name):
        self.df = pd.read_csv(file_name)
        self.df = self.df.astype({'category': 'category', 'short_description': 'str'})
        self.df['num_cat'] = self.df['category'].cat.codes

        self.sentences = self.df['short_description']
        self.labels = self.df['num_cat']

        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None

    def clean_data(self):
        whitespace = re.compile(r"\s+")
        user = re.compile(r"(?i)@[a-z0-9_]+")

        for i in range(len(self.sentences)):
            text = self.sentences[i]
            text = whitespace.sub(' ', text)
            text = user.sub('', text)
            text = re.sub(r"\[[^()]*\]","", text)
            text = re.sub("\d+", "", text)
            text = re.sub(r'[^\w\s]','',text)
            text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
            text = text.lower()
        
            # Removing stop-words
            text = [word for word in text.split() if word not in STOPWORDS]
        
            # Word lemmatization
            lemmatizer = WordNetLemmatizer()
            text = [lemmatizer.lemmatize(word, 'v') for word in text]
            
            self.sentences.iloc[i] = ' '.join(text)
    
    def get_train(self):
        return zip(self.y_train, self.x_train)
    
    def get_test(self):
        return zip(self.y_test, self.x_test)
    
    def split_test_train(self):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.sentences, self.labels, test_size=0.20)

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]


In [4]:
huffPo = MyDataset('/common/users/shared/cs543_fall22_group3/huffpo/lemmatized.csv')
huffPo.split_test_train()

In [6]:
tokenizer = get_tokenizer('basic_english')
train_iter = huffPo.get_train()

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [7]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [8]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = torch.cat(text_list)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = huffPo.get_train()
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [80]:
class TweetClassifier(nn.Module):
    def __init__(self, vocab_size=45, embed_dim=64, num_layers=2, num_classes=42):
        super(TweetClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_classes = num_classes

        self.embedding = nn.EmbeddingBag(self.vocab_size, self.embed_dim, sparse=True)
        self.lstm = nn.LSTM(input_size=self.embed_dim, hidden_size=self.embed_dim, num_layers=self.num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(embed_dim, self.num_classes)

    def forward(self, text, offsets):
        h = torch.zeros((self.LSTM_layers, self.embed_dim)).to(device)
        c = torch.zeros((self.LSTM_layers, self.embed_dim)).to(device)

        nn.init.xavier_normal_(h)
        nn.init.xavier_normal_(c)

        x = self.embedding(text, offsets)
        x, (h, c) = self.lstm(x, (h, c))
        x = self.dropout(x)
        x = self.fc(x)

        return x


In [81]:
train_iter = huffPo.get_train()
vocab_size = len(vocab)
num_classes = len(set([label for (label, text) in train_iter]))
model = TweetClassifier(vocab_size=vocab_size, num_classes=num_classes).to(device)

In [82]:
def train(dataloader, model, criterion, optimizer, epoch):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(f"| epoch {epoch:3d} | {idx:5d}/{len(dataloader):5d} batches | accuracy {total_acc/total_count:8.3f}")
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    
    return total_acc / total_count

In [ ]:
# Hyperparameters
EPOCHS = 50 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = huffPo.get_train()
test_iter = huffPo.get_test()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    
    print('-' * 59)
    print(f'| end of epoch {epoch:3d} | time: {time.time() - epoch_start_time:5.2f}s | valid accuracy {accu_val:8.3f}')
    print('-' * 59)

| epoch   1 |   500/ 2489 batches | accuracy    0.168
| epoch   1 |  1000/ 2489 batches | accuracy    0.174
| epoch   1 |  1500/ 2489 batches | accuracy    0.197
| epoch   1 |  2000/ 2489 batches | accuracy    0.221
-----------------------------------------------------------
| end of epoch   1 | time:  6.94s | valid accuracy    0.241 
-----------------------------------------------------------
| epoch   2 |   500/ 2489 batches | accuracy    0.251
| epoch   2 |  1000/ 2489 batches | accuracy    0.262
| epoch   2 |  1500/ 2489 batches | accuracy    0.273
| epoch   2 |  2000/ 2489 batches | accuracy    0.285
-----------------------------------------------------------
| end of epoch   2 | time:  6.99s | valid accuracy    0.301 
-----------------------------------------------------------
| epoch   3 |   500/ 2489 batches | accuracy    0.311
| epoch   3 |  1000/ 2489 batches | accuracy    0.319
| epoch   3 |  1500/ 2489 batches | accuracy    0.321
| epoch   3 |  2000/ 2489 batches | accuracy

In [75]:
print(f'Test accuracy is: {evaluate(test_dataloader)}')

0.3991313892998616
